In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('../Style.mplstyle')
import pandas as pd
import os
import sys
import scipy
from iminuit import Minuit
from scipy import stats
sys.path.append('..')
from read_data_functions import *
from plot_functions import *
from calculations import *

In [8]:
parent_path = '../../../../'
path = 'L:/PG-Nanoteknologi/PROJEKTER/2024 Laura og Nan/Lund/'

SMPS = read_SMPS(path, parent_path, [0, 0])
OPS_APS = read_OPS(path, parent_path, 0)

NS_dict_keys = ['2024-10-15_NanoScan', '2024-10-16_NanoScan', '2024-10-17_NanoScan']
NS_bins = SMPS['2024-10-21_NanoScan'].keys()[3:16].to_list()

NS_bg_mean, NS_bg_std, NS_bg_error = bin_mean(['2024/10/21 11:45:00', '2024/10/21 14:10:00'], 
                                              SMPS['2024-10-21_NanoScan'], NS_bins, 'Time', 0.1)

for key in NS_dict_keys:
    df = SMPS[key]
    for i, bin in enumerate(NS_bins):
        df[bin] = df[bin] - NS_bg_mean[i]

    SMPS[key] = df

dict_keys = ['Exp1', 'Exp2', 'Exp3', 'Exp4']
OPS_mid_point = [0.337, 0.419, 0.522, 0.650, 0.809, 1.007, 1.254, 1.562, 1.944, 2.421, 3.014, 3.752, 4.672, 5.816, 7.241, 9.015]
OPS_bins = OPS_APS['Exp1'].keys()[1:17].to_list()

for dict_key in dict_keys:
    for old_key, new_key in zip(OPS_bins, OPS_mid_point):
        OPS_APS[dict_key] = OPS_APS[dict_key].rename(columns = {old_key: new_key})

OPS_cut_point = [0.300, 0.374, 0.465, 0.579, 0.721, 0.897, 1.117, 1.391, 1.732, 2.156, 2.685, 3.343, 4.162, 5.182, 6.451, 8.031, 10.000]

NS_OPS_cut = [0.010, 0.0132, 0.0179, 0.0234, 0.032, 0.0416, 0.057, 0.0739, 0.1015, 0.1314, 0.1805, 0.2337, 
              0.300, 0.374, 0.465, 0.579, 0.721, 0.897, 1.117, 1.391, 1.732, 2.156, 2.685, 3.343, 4.162, 5.182, 6.451, 8.031, 10.000]

time_wBG = [['2024-10-16 09:50:00', '2024-10-16 12:50:00'],
            ['2024-10-16 14:30:00', '2024-10-16 16:30:00'],
            ['2024-10-17 09:30:00', '2024-10-17 12:10:00'],
            ['2024-10-17 14:10:00', '2024-10-17 15:30:00']]

timestamps_exp = [['2024-10-16 10:45:00', '2024-10-16 12:26:00'],
                  ['2024-10-16 15:00:00', '2024-10-16 15:34:00'],
                  ['2024-10-17 10:20:00', '2024-10-17 12:00:00'],
                  ['2024-10-17 14:30:00', '2024-10-17 15:00:00']]

NanoScan_dict_keys = ['2024-10-16_NanoScan', '2024-10-16_NanoScan', '2024-10-17_NanoScan', '2024-10-17_NanoScan']
NS_OPS_number, NS_OPS_mass, NS_OPS_keys, NS_OPS_bin_mean = merge_data(SMPS, NanoScan_dict_keys, [3, 15], OPS_APS, dict_keys, [1, 17], time_wBG, ['Time', 'Time'], False, True)

Failed to read file with separation: 	
Failed to read file with separation: ,
Failed to read file with separation: ,


In [ ]:
# Calculate bin means

In [ ]:
def gauss(x, p, mu, sigma):
    return p*stats.norm.pdf(x, mu, sigma)

def d_gauss(x, p1, p2, mu1, sigma1, mu2, sigma2):
    return p1*stats.norm.pdf(x, mu1, sigma1) + p2*stats.norm.pdf(x, mu2, sigma2)

def t_gauss(x, p1, p2, p3, mu1, sigma1, mu2, sigma2, mu3, sigma3):
    return p1*stats.norm.pdf(x, mu1, sigma1) + p2*stats.norm.pdf(x, mu2, sigma2) + p3*stats.norm.pdf(x, mu3, sigma3)

def dgauss_fit(x, y, ey, guess):

    Npoints = len(y)
    # Make sure data are arrays
    x, y, ey = np.array(x), np.array(y), np.array(ey)

    # Define Chi2 calculation
    def chi2(p1, p2, mu1, sigma1, mu2, sigma2) :
        y_fit = d_gauss(x, p1, p2, mu1, sigma1, mu2, sigma2)
        chi2 = np.sum(((y - y_fit) / ey)**2)
        return chi2
    chi2.errordef = 1.0

    # Here we let Minuit know, what to minimise, how, and with what starting parameters
    minuit = Minuit(chi2, p1 = guess[0], p2 = guess[1], mu1 = guess[2], sigma1 = guess[3], mu2 = guess[4], sigma2 = guess[5])

    # Perform the actual fit
    minuit.migrad();

    # # Extract the fitting parameters and their errors
    # p1_fit, ep1_fit = minuit.values['p1'], minuit.errors['p1']
    # p2_fit, ep2_fit = minuit.values['p2'], minuit.errors['p2']
    # mu1_fit, emu1_fit = minuit.values['mu1'], minuit.errors['mu1']
    # sigma1_fit, esigme1_fit = minuit.values['sigma1'], minuit.errors['sigma1']
    # mu2_fit, emu2_fit = minuit.values['mu2'], minuit.errors['mu2']
    # sigma2_fit, esigme2_fit = minuit.values['sigma2'], minuit.errors['sigma2']

    Nvar = len(guess)           # Number of variables
    Ndof_fit = Npoints - Nvar   # Number of degrees of freedom

    # Get the minimal value obtained for the quantity to be minimised (here the Chi2)
    Chi2_fit = minuit.fval                          # The chi2 value
    Prob_fit = stats.chi2.sf(Chi2_fit, Ndof_fit)    # The chi2 probability given N degrees of freedom

    return minuit.values, minuit.errors, Ndof_fit, Chi2_fit, Prob_fit

def plot_fitted_mean(ax, timestamps, df, df_keys, timelabel, inst_error, bin_means, bin_edges, guess, gauss_type):

    mean_number, error_number, mean_mass, error_mass, ax, ax2 = plot_bin_mean()

    if gauss_type == 'double':
        fit_params, fit_errors = dgauss_fit(bin_means, mean_number, error_number, guess)
    return

In [ ]:
# if gauss_type == 'triple':
#         def fit_function(x, p1, p2, p3, mu1, sigma1, mu2, sigma2, mu3, sigma3):
#             return (p1)*stats.norm.pdf(x, mu1, sigma1) + p2*stats.norm.pdf(x, mu2, sigma2) + p3*stats.norm.pdf(x, mu3, sigma3)
    
#         # Define Chi2 calculation
#         def chi2(p1, p2, p3, mu1, sigma1, mu2, sigma2, mu3, sigma3) :
#             y_fit = fit_function(x, p1, p2, p3, mu1, sigma1, mu2, sigma2, mu3, sigma3)
#             chi2 = np.sum(((y - y_fit) / ey)**2)
#             return chi2
#         chi2.errordef = 1.0

#         # Here we let Minuit know, what to minimise, how, and with what starting parameters:   
#         minuit = Minuit(chi2, p1 = guess[0], p2 = guess[1], p3 = guess[2], mu1 = guess[3], sigma1 = guess[4], mu2 = guess[5], sigma2 = guess[6], mu3 = guess[7], sigma3 = guess[8])